In [8]:
from langchain.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from PyPDF2 import PdfReader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os
import fitz
from langchain_experimental.text_splitter import SemanticChunker
from langchain.schema import Document

In [12]:
# Directorio para cargar los archivos PDF
pdf_directory = './doc'  # Ajusta la ruta según tu estructura de directorios
os.makedirs(pdf_directory, exist_ok=True)

# Función para verificar si un archivo es PDF
def is_pdf(file_path):
    try:
        with open(file_path, 'rb') as file:
            # Verificar si el archivo empieza con la cabecera %PDF
            header = file.read(4)
            if header != b'%PDF':
                return False

        # Intentar abrir el archivo con PyMuPDF
        doc = fitz.open(file_path)
        doc.close()
        return True
    except (fitz.FitzError, IOError):
        return False

# Directorio de entrada que contiene archivos PDF
input_directory = './doc'  # Cambia esta ruta si es diferente

# Mover y validar los archivos cargados
valid_pdf_files = []
for filename in os.listdir(input_directory):
    if filename.lower().endswith('.pdf'):
        file_path = os.path.join(input_directory, filename)
        print(f"Cargando archivo: {file_path}")

        # Mover el archivo al directorio especificado
        destination_path = os.path.join(pdf_directory, filename)

        # Mover el archivo al directorio especificado
        os.rename(file_path, destination_path)

        # Validar si es un PDF
        if is_pdf(destination_path):
            valid_pdf_files.append(filename)
        else:
            print(f"{filename} no es un archivo PDF válido y será ignorado.")

# Imprimir archivos válidos
print("Archivos PDF válidos:", valid_pdf_files)

Cargando archivo: ./doc\preguntas.pdf
Cargando archivo: ./doc\respuestas.pdf
Cargando archivo: ./doc\teoria.pdf
Archivos PDF válidos: ['preguntas.pdf', 'respuestas.pdf', 'teoria.pdf']


In [15]:
# Directorio que contiene los archivos PDF
pdf_directory = './doc'

# Lista para almacenar todos los documentos
all_docs = []

# Iterar sobre todos los archivos en el directorio
for filename in os.listdir(pdf_directory):
    if filename.lower().endswith('.pdf'):
        file_path = os.path.join(pdf_directory, filename)
        print(f"Cargando archivo: {file_path}")

        try:
            # Cargar el PDF
            loader = PyPDFLoader(file_path)
            docs = loader.load()

            # Agregar los documentos cargados a la lista
            all_docs.extend(docs)
            print(f"Archivo {filename} cargado exitosamente.")
        except Exception as e:
            print(f"Error al cargar {filename}: {e}")

# Imprimir la cantidad total de partes del PDF
print(f"Cantidad total de partes del PDF: {len(all_docs)}")

Cargando archivo: ./doc\preguntas.pdf
Archivo preguntas.pdf cargado exitosamente.
Cargando archivo: ./doc\respuestas.pdf
Archivo respuestas.pdf cargado exitosamente.
Cargando archivo: ./doc\teoria.pdf
Archivo teoria.pdf cargado exitosamente.
Cantidad total de partes del PDF: 53


In [18]:
# Crear el modelo de embeddings
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

# Crear el Semantic Chunker
semantic_chunker = SemanticChunker(embeddings, breakpoint_threshold_type="percentile")

# Dividir los documentos en partes semánticas manejables
splits = []
for doc in all_docs:
    chunks = semantic_chunker.split_text(doc.page_content)
    for chunk in chunks:
        # Crear un objeto Document para cada fragmento
        splits.append(Document(page_content=chunk, metadata=doc.metadata))

# Crear el vector store y almacenar los vectores
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings, persist_directory='./vectordb')

# Configurar el recuperador
retriever = vectorstore.as_retriever()

print("Vector store y recuperador creados exitosamente.")

Vector store y recuperador creados exitosamente.


In [ ]:
# embeddings = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2') #modelo de embeddings gratis

# text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50) #usamos menos tokens que en el anterior debido a que este modelo es inferior
# splits = text_splitter.split_documents(all_docs)
# vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings, persist_directory='./vectordb')

# retriever = vectorstore.as_retriever()

In [19]:
retriever.get_relevant_documents('¿Cuál es la idea básica del proyecto?')

c:\Users\Pablo\Desktop\RAG\env\Lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


[Document(metadata={'page': 41, 'source': './doc\\teoria.pdf'}, page_content='¿Qué es?'),
 Document(metadata={'page': 0, 'source': './doc\\respuestas.pdf'}, page_content='1. Presentación\ndel\nProyecto\n1.1. Descripción\ndel\nNegocio\nAyuda\npara\nResponder:\n●\nIdea\nBásica\ndel\nProyecto:\no\nPiensa\nen\nqué\noportunidad\nde\nmercado\nestás\naprovechando\ncon\ntu\nidea\nde \nnegocio. Según\nel\ndocumento,\nla\nidea\ndebe\ncubrir\nuna\ndemanda\nno \nsatisfecha\no\nmejorar\nun\nproducto/servicio\nexistente. o\nDescribe\nel\nproducto\no\nservicio\ndestacando\nsi\nes\nalgo\nnuevo\nen\nel \nmercado\no\nuna\ninnovación\nsobre\nalgo\nya\nexistente. ●\nProductos\no\nServicios:\no\nSi\nes\nun\nproducto,\ndescribe\nsus\ncaracterísticas,\nutilidad\ny\nsi\nes\nalgo \nnovedoso\no\nconocido. Explica\ncómo\nfuncionará\ny\ncuáles\nson\nsus\nventajas \ncompetitivas. o\nSi\nes\nun\nservicio,\ndetalla\nlo\nque\nse\nva\na\nhacer ,\ncómo\nse\nhará\ny\nqué \nbeneficios\naportará\nal\ncliente. ●\nRazones\n

In [ ]:
# import shutil

# # Ruta de la carpeta que deseas comprimir
# folder_path = '/content/vectordb'  # Cambia esto a la ruta de tu carpeta
# zip_path = '/content/vectordb.zip'

# # Comprimir la carpeta
# shutil.make_archive('/content/vectordb', 'zip', folder_path)